In [1]:
import pandas as pd
import numpy as np

In [2]:
videos_df = pd.read_csv("videos_df.csv")

In [3]:
socialblade = pd.read_csv("socialblade_df.csv")

In [4]:
socialblade.head()

,Channel Name,Channel ID,Category
0,Top Gear,UCjOl2AUblVmg2rA_cRgZkFg,AUTO & VEHICLES
1,ChrisFix,UCes1EvRjcKU4sY_UEavndBw,AUTO & VEHICLES
2,MotorTrend Channel,UCsAegdhiYLEoaFGuJFVrqFQ,AUTO & VEHICLES
3,carwow,UCUhFaUpnq31m6TNX2VKVSVA,AUTO & VEHICLES
4,Supercar Blondie,UCKSVUHI9rbbkXhvAXK-2uxA,AUTO & VEHICLES


In [5]:
socialblade = socialblade.rename(columns={'Channel ID': 'channelId'})

In [6]:
videos_df.head()

,channelId,description,publishedAt,videoId,thumbnails,videoTitle,commentCount,dislikeCount,likeCount,viewCount
0,UCjOl2AUblVmg2rA_cRgZkFg,"In this week's Top Gear, Flintoff gets his han...",2020-10-09 09:35:54,NXX338WY_Lw,https://i.ytimg.com/vi/NXX338WY_Lw/default.jpg,PREVIEW: Attempting 200mph in the Jaguar XJ220...,528.0,257.0,5819.0,184447.0
1,UCjOl2AUblVmg2rA_cRgZkFg,"From the humble new Volkswagen GTI, right down...",2020-10-09 11:21:23,dtHcdU2c71Y,https://i.ytimg.com/vi/dtHcdU2c71Y/default.jpg,Which car will win Top Gear Speed Week 2020? (...,568.0,273.0,7136.0,217619.0
2,UCjOl2AUblVmg2rA_cRgZkFg,Here's Chris Harris' take on the rocket-disgui...,2020-10-07 07:40:22,vnrtWe-RAzg,https://i.ytimg.com/vi/vnrtWe-RAzg/default.jpg,Chris Harris on... the Ferrari SF90 Stradale |...,1091.0,408.0,10189.0,437777.0
3,UCjOl2AUblVmg2rA_cRgZkFg,"16 contenders, 8,553bhp and a festival to reme...",2020-10-06 13:59:38,Ra1F0TsOCPs,https://i.ytimg.com/vi/Ra1F0TsOCPs/default.jpg,Chris Harris vs 2020’s Best Performance Cars |...,579.0,202.0,7126.0,191070.0
4,UCjOl2AUblVmg2rA_cRgZkFg,"The 986bhp Ferrari SF90 is, unsurprisingly, no...",2020-10-06 07:36:13,fXysipmTxcQ,https://i.ytimg.com/vi/fXysipmTxcQ/default.jpg,FASTEST TOP GEAR LAP? Ferrari SF90 Stiglap | T...,888.0,168.0,9697.0,572569.0


In [7]:
videos = pd.merge(socialblade, videos_df, on='channelId')

In [8]:
videos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408690 entries, 0 to 408689
Data columns (total 12 columns):
Channel Name    408690 non-null object
channelId       408690 non-null object
Category        408690 non-null object
description     406594 non-null object
publishedAt     408690 non-null object
videoId         408690 non-null object
thumbnails      408675 non-null object
videoTitle      408690 non-null object
commentCount    392556 non-null float64
dislikeCount    407143 non-null float64
likeCount       407143 non-null float64
viewCount       408583 non-null float64
dtypes: float64(4), object(8)
memory usage: 40.5+ MB


In [9]:
videos1 = videos.loc[(videos['likeCount'].notnull() & videos['dislikeCount'].notnull() & videos['viewCount'].notnull())]

In [10]:
#check null values, need to fill those
print("null comments")
print(len(videos1.loc[videos1['commentCount'].isna()]))


null comments
15554


Mainly predicting comment counts

In [11]:
len(videos1)

407093

In [12]:
# get a list of categories
cat=videos1['Category'].unique().tolist()
cat

['AUTO & VEHICLES',
 'COMEDY',
 'EDUCATION',
 'ENTERTAINMENT',
 'FILM',
 'GAMING',
 'HOW TO & STYLE',
 'MADE FOR KIDS',
 'MUSIC',
 'NEWS & POLITICS',
 'NONPROFIT & ACTIVISM',
 'PEOPLE & BLOGS',
 'PETS & ANIMALS',
 'SCIENCE & TECHNOLOGY',
 'SPORTS',
 'TRAVEL']

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
seed = 123

### Regression on Videos

In [14]:
noNull = videos1[videos1['commentCount'].notnull()] #remove null from comments for training data
commentX = noNull[['viewCount','dislikeCount','likeCount']]
commentY = noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)

(313231, 3) (78308, 3) (313231,) (78308,)


In [15]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = np.maximum(0, model.predict(test))
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)

Root means score 7345.495417002755
R^2 : 0.17800195919791728
        True Labels  Predicted Labels
329420         74.0        401.024611
329421        570.0       1290.860828
329422         47.0        299.053212
329423         87.0        412.919105
329424         75.0        336.722431
...             ...               ...
408685         14.0        266.597516
408686          9.0        261.749835
408687          8.0        263.055681
408688        151.0        278.400628
408689         24.0        265.219457

[78308 rows x 2 columns]


The model has a high mean squared error and low R-squared score, thus it is not a good prediction model.

In [16]:
#with zeros
commentZero = np.array([0]*len(y_pred))

In [17]:
print('Root means score', np.sqrt(mean_squared_error(y_test, commentZero)))
print('R^2: %.2f' % r2_score(y_test, commentZero))
#print("R^2:",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels':commentZero}
SK = pd.DataFrame(data = d1)
print(SK)

Root means score 8182.8456325873485
R^2: -0.02
        True Labels  Predicted Labels
329420         74.0                 0
329421        570.0                 0
329422         47.0                 0
329423         87.0                 0
329424         75.0                 0
...             ...               ...
408685         14.0                 0
408686          9.0                 0
408687          8.0                 0
408688        151.0                 0
408689         24.0                 0

[78308 rows x 2 columns]


In [18]:
#with average
avg = sum(noNull['commentCount'])/len(videos1)
commentAvg = np.array([avg]*len(y_pred))
avg

1785.1573399689996

In [19]:
print('Root means score', np.sqrt(mean_squared_error(y_test, commentAvg)))
print('R^2: %.2f' % r2_score(y_test, commentAvg))
#print("R^2:",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels':commentAvg}
SK = pd.DataFrame(data = d1)
print(SK)

Root means score 8131.525071335478
R^2: -0.01
        True Labels  Predicted Labels
329420         74.0        1785.15734
329421        570.0        1785.15734
329422         47.0        1785.15734
329423         87.0        1785.15734
329424         75.0        1785.15734
...             ...               ...
408685         14.0        1785.15734
408686          9.0        1785.15734
408687          8.0        1785.15734
408688        151.0        1785.15734
408689         24.0        1785.15734

[78308 rows x 2 columns]


By imputing the null values with 0 or mean yields low R^2 score and high mean sqaured error

### Auto and Vehicles

In [20]:
auto = videos1.loc[videos1['Category']=='AUTO & VEHICLES']

In [21]:
#check null values, need to fill those
print("null comments")
print(len(auto.loc[auto['commentCount'].isna()]))

null comments
12


In [22]:
#fill view - start with the least missing values

auto_noNull = auto.loc[auto['commentCount'].notnull()] #remove null from comments for training data
commentX = auto_noNull[['viewCount','dislikeCount','likeCount']]
commentY = auto_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)

(5788, 3) (1448, 3) (5788,) (1448,)


In [23]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)

Root means score 1673.0886811288349
R^2 : 0.41653638193753567
      True Labels  Predicted Labels
5807        839.0       1047.699479
5808        868.0       1116.628477
5809        889.0       1101.461168
5810       1526.0       1136.672749
5811       2104.0       1492.792444
...           ...               ...
7250       5626.0       3291.985363
7251       1786.0       1621.026273
7252       2252.0       1860.857573
7253       1592.0       1764.879730
7254       2750.0       1985.828117

[1448 rows x 2 columns]


In [24]:
est = sm.OLS(y_train, train)
est2 = est.fit()
print(est2.summary())

                                 OLS Regression Results                                
Dep. Variable:           commentCount   R-squared (uncentered):                   0.748
Model:                            OLS   Adj. R-squared (uncentered):              0.748
Method:                 Least Squares   F-statistic:                              5732.
Date:                Wed, 28 Oct 2020   Prob (F-statistic):                        0.00
Time:                        11:41:03   Log-Likelihood:                         -52440.
No. Observations:                5788   AIC:                                  1.049e+05
Df Residuals:                    5785   BIC:                                  1.049e+05
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

All p-values < 0.05, significant

The difference between the 2 models:
- In the OLS model you are using the training data to fit and predict.
- With the LinearRegression model you are using training data to fit and test data to predict, therefore different results in R2 scores.

In [25]:
#predict missing comment count
missingAuto = auto.loc[auto['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingAuto[['viewCount','dislikeCount','likeCount']]))

In [26]:
#fill auto 
auto.loc[auto.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### Comedy

In [27]:
comedy = videos1.loc[videos1['Category']=='COMEDY']

In [28]:
#check null values, need to fill those
print("null comments")
print(len(comedy.loc[comedy['commentCount'].isna()]))


null comments
5


In [29]:
#comment
comedy_noNull = comedy.loc[comedy['commentCount'].notnull()]
commentX = comedy_noNull[['viewCount','dislikeCount','likeCount']]
commentY = comedy_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False)
print(train.shape,test.shape,y_train.shape,y_test.shape)

(7492, 3) (1873, 3) (7492,) (1873,)


In [30]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
#print('Variance score: %.2f' % r2_score(y_test, y_pred))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)

Root means score 17374.590156368948
R^2 : 0.5636174867723003
       True Labels  Predicted Labels
14750      27318.0      20971.731572
14751       8697.0       8619.763247
14752      13096.0      12519.716843
14753      12139.0      11351.800136
14754       6595.0       6085.620045
...            ...               ...
16628       5162.0      10812.935924
16629       2971.0       4826.312652
16630      12899.0       8131.266767
16631       2319.0       5295.574735
16632       5171.0       5926.027047

[1873 rows x 2 columns]


In [31]:
missingComCom = comedy.loc[comedy['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingComCom[['viewCount','dislikeCount','likeCount']]))
comedy.loc[comedy.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [32]:
comedy.loc[comedy['commentCount'].isna()]

,Channel Name,channelId,Category,description,publishedAt,videoId,thumbnails,videoTitle,commentCount,dislikeCount,likeCount,viewCount


### Education

In [33]:
edu = videos1.loc[videos1['Category']=='EDUCATION']

In [34]:
#check null values, need to fill those
print("null comments")
print(len(edu.loc[edu['commentCount'].isna()]))


null comments
2316


In [35]:
edu_noNull = edu.loc[edu['commentCount'].notnull()]
commentX = edu_noNull[['viewCount','dislikeCount','likeCount']]
commentY = edu_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(5075, 3) (1269, 3) (5075,) (1269,)


In [36]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
#print('Variance score: %.2f' % r2_score(y_test, y_pred))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 4151.703106527555
R^2 : 0.8006313492879549
       True Labels  Predicted Labels
24134        263.0       1667.771159
24135         37.0       1731.255461
24136         16.0       1907.943915
24137          3.0        595.630834
24138        107.0        423.365772
...            ...               ...
25522       6924.0       6041.973261
25523       4057.0       5587.448234
25524       6060.0       8865.696882
25525      17760.0       7692.191629
25526       5847.0       7980.133308

[1269 rows x 2 columns]


In [37]:
#fill missig values
missingEdu = edu.loc[edu['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingEdu[['viewCount','dislikeCount','likeCount']]))
edu.loc[edu.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [38]:
edu.loc[edu['commentCount'].isna()]

,Channel Name,channelId,Category,description,publishedAt,videoId,thumbnails,videoTitle,commentCount,dislikeCount,likeCount,viewCount


### ENTERTAINMENT

In [39]:
ent = videos1.loc[videos1['Category']=='ENTERTAINMENT']

In [40]:
#check null values, need to fill those
print("null comments")
print(len(ent.loc[ent['commentCount'].isna()]))

null comments
4918


In [41]:
ent_noNull = ent.loc[ ent['commentCount'].notnull()]
commentX = ent_noNull[['viewCount','dislikeCount','likeCount']]
commentY = ent_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(8843, 3) (2211, 3) (8843,) (2211,)


In [42]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
#print('Variance score: %.2f' % r2_score(y_test, y_pred))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 11746.952047195382
R^2 : 0.5412613249640561
       True Labels  Predicted Labels
38064        869.0         15.715279
38065       1048.0        483.529664
38066        578.0        189.695063
38067       1651.0        923.290063
38068       1327.0       2657.920079
...            ...               ...
41150          0.0       1650.483952
41151          0.0       7828.434270
41154          0.0       2080.553403
41155          0.0       2636.049805
41156          0.0        473.989380

[2211 rows x 2 columns]


In [43]:
#fill missing values
missingEnt = ent.loc[ent['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingEnt[['viewCount','dislikeCount','likeCount']]))
np.maximum(0, model.predict(test))
ent.loc[ent.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [44]:
ent.loc[ent['commentCount'].isna()]

,Channel Name,channelId,Category,description,publishedAt,videoId,thumbnails,videoTitle,commentCount,dislikeCount,likeCount,viewCount


In [45]:
ent.loc[ent.commentCount < 0]

,Channel Name,channelId,Category,description,publishedAt,videoId,thumbnails,videoTitle,commentCount,dislikeCount,likeCount,viewCount


### FILM

In [46]:
film = videos1.loc[videos1['Category']=='FILM']

In [47]:
#check null values, need to fill those
print("null comments")
print(len(film.loc[film['commentCount'].isna()]))

null comments
1778


In [48]:
film_noNull = film.loc[film['commentCount'].notnull()]
commentX = film_noNull[['viewCount','dislikeCount','likeCount']]
commentY = film_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(23268, 3) (5818, 3) (23268,) (5818,)


In [49]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
#print('Variance score: %.2f' % r2_score(y_test, y_pred))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 2095.0462763369574
R^2 : 0.38636404074352015
       True Labels  Predicted Labels
65741          4.0        130.152763
65742         38.0        167.779003
65743          2.0        265.722027
65744          0.0         63.307902
65745          1.0         78.450789
...            ...               ...
73321       2969.0        988.622553
73322       5647.0       4599.631757
73323       3977.0       3010.138043
73324       1628.0        801.082738
73325       1990.0        719.906402

[5818 rows x 2 columns]


In [50]:
#fill missing values
missingFilm = film.loc[film['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingFilm[['viewCount','dislikeCount','likeCount']]))
film.loc[film.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [51]:
film.loc[film['commentCount'].isna()]

,Channel Name,channelId,Category,description,publishedAt,videoId,thumbnails,videoTitle,commentCount,dislikeCount,likeCount,viewCount


### GAMING


In [52]:
game = videos1.loc[videos1['Category']=='GAMING']

In [53]:
#check null values, need to fill those
print("null comments")
print(len(game.loc[game['commentCount'].isna()]))

null comments
28


In [54]:
game_noNull = game.loc[game['commentCount'].notnull()]
commentX = game_noNull[['viewCount','dislikeCount','likeCount']]
commentY = game_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(16212, 3) (4054, 3) (16212,) (4054,)


In [55]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
#print('Variance score: %.2f' % r2_score(y_test, y_pred))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 12705.26209130222
R^2 : 0.027019511105150325
       True Labels  Predicted Labels
89561       6008.0       8604.795165
89562       3315.0       5613.745333
89563       3371.0       6238.387046
89564       4173.0       6423.895659
89565       3344.0       5447.792618
...            ...               ...
93615        767.0       3399.381836
93616        699.0       3884.492958
93617        706.0       3516.440891
93618       2464.0       3967.512319
93619        426.0       3061.067173

[4054 rows x 2 columns]


In [56]:
#fill missing values
missingGame = game.loc[game['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingGame[['viewCount','dislikeCount','likeCount']]))
game.loc[game.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [57]:
game.loc[game['commentCount'].isna()]

,Channel Name,channelId,Category,description,publishedAt,videoId,thumbnails,videoTitle,commentCount,dislikeCount,likeCount,viewCount


### HOW TO & STYLE

In [58]:
how = videos1.loc[videos1['Category']=='HOW TO & STYLE']

In [59]:
#check null values, need to fill those
print("null comments")
print(len(how.loc[how['commentCount'].isna()]))

null comments
18


In [60]:
how_noNull = how.loc[how['commentCount'].notnull()]
commentX = how_noNull[['viewCount','dislikeCount','likeCount']]
commentY = how_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(13556, 3) (3389, 3) (13556,) (3389,)


In [61]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
#print('Variance score: %.2f' % r2_score(y_test, y_pred))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 24031.225071931694
R^2 : 0.19187259244458177
        True Labels  Predicted Labels
107177         88.0        199.484786
107178         35.0         84.176222
107179         66.0        128.555154
107180       1023.0       3888.234494
107181        174.0        598.557413
...             ...               ...
110580        532.0        454.801327
110581       2428.0       2995.631152
110582        921.0       1102.068329
110583        812.0        559.547705
110584       1149.0        393.893633

[3389 rows x 2 columns]


In [62]:
#fill missing values
missingHow = how.loc[how['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingHow[['viewCount','dislikeCount','likeCount']]))
how.loc[how.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### MADE FOR KIDS

In [63]:
kids = videos1.loc[videos1['Category']=='MADE FOR KIDS']

In [64]:
#check null values, need to fill those
print("null comments")
print(len(kids.loc[kids['commentCount'].isna()]))

null comments
2755


In [65]:
kids_noNull = kids.loc[kids['commentCount'].notnull()]
commentX = kids_noNull[['viewCount','dislikeCount','likeCount']]
commentY = kids_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(4014, 3) (1004, 3) (4014,) (1004,)


In [66]:
est = sm.OLS(y_train, train)
est2 = est.fit()
print(est2.summary())

                                 OLS Regression Results                                
Dep. Variable:           commentCount   R-squared (uncentered):                   0.827
Model:                            OLS   Adj. R-squared (uncentered):              0.827
Method:                 Least Squares   F-statistic:                              6406.
Date:                Wed, 28 Oct 2020   Prob (F-statistic):                        0.00
Time:                        11:41:04   Log-Likelihood:                         -30419.
No. Observations:                4014   AIC:                                  6.084e+04
Df Residuals:                    4011   BIC:                                  6.086e+04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

In [67]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
#print('Variance score: %.2f' % r2_score(y_test, y_pred))
print("R^2:",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 373.70693847401395
R^2: -0.9922933055043289
        True Labels  Predicted Labels
117051          7.0        145.239352
117052         93.0         57.811803
117053         13.0         58.856678
117054         15.0         58.281161
117055         64.0        511.437327
...             ...               ...
118056          2.0         10.868541
118057          2.0         16.027439
118058         21.0        206.233056
118059          0.0         10.220877
118060          6.0         29.305825

[1004 rows x 2 columns]


A negative R^2 means that the chosen model fits worse than a horizontal line, it does not follow the trend of the data

In [68]:
average = sum(kids_noNull['commentCount'])/len(kids_noNull['commentCount'])

In [69]:
commentAvg = np.array([average]*len(y_pred))
commentAvg

array([124.56895177, 124.56895177, 124.56895177, ..., 124.56895177,
       124.56895177, 124.56895177])

In [70]:
print('Root means score', np.sqrt(mean_squared_error(y_test, commentAvg)))
print('Variance score: %.2f' % r2_score(y_test, commentAvg))
#print("R^2:",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels':average}

Root means score 266.5841606947479
Variance score: -0.01


By using the average comment count, the mean squared error and variance is lower. Thus for this category, the average comment count will be used instead.

In [71]:
 len(kids.loc[kids['commentCount'].isna()])

2755

In [72]:
commentAvg = np.array([average]* len(kids.loc[kids['commentCount'].isna()]))

In [73]:
#fill missing values
missingKids = kids.loc[kids['commentCount'].isna()]
#pred = model.predict(missingKids[['viewCount','dislikeCount','likeCount']])
kids.loc[kids.commentCount.isnull(), 'commentCount'] = commentAvg

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### MUSIC

In [74]:
music = videos1.loc[videos1['Category']=='MUSIC']

In [75]:
#check null values, need to fill those
print("null comments")
print(len(music.loc[music['commentCount'].isna()]))

null comments
4


In [76]:
music_noNull = music.loc[music['commentCount'].notnull()]
commentX = music_noNull[['viewCount','dislikeCount','likeCount']]
commentY = music_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(1079, 3) (270, 3) (1079,) (270,)


In [77]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
#print('Variance score: %.2f' % r2_score(y_test, y_pred))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 54903.378941947696
R^2 : 0.5449468281846672
        True Labels  Predicted Labels
119479       4568.0       4971.910973
119480      44502.0      97708.192381
119481       6022.0      18159.172557
119482       2035.0      -3382.581303
119483       3635.0       4279.045455
...             ...               ...
119745        312.0      -4660.413704
119746       1386.0      -3839.590697
119747        339.0      -4592.463438
119748        540.0      -4369.228625
119749        712.0      -4430.992268

[270 rows x 2 columns]


In [78]:
#fill missing values
missingMusic = music.loc[music['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingMusic[['viewCount','dislikeCount','likeCount']]))
music.loc[music.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### NEWS & POLITICS

In [79]:
news = videos1.loc[videos1['Category']=='NEWS & POLITICS']

In [80]:
#check null values, need to fill those
print("null comments")
print(len(news.loc[news['commentCount'].isna()]))

null comments
1430


In [81]:
news_noNull = news.loc[news['commentCount'].notnull()]
commentX = news_noNull[['viewCount','dislikeCount','likeCount']]
commentY = news_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(133079, 3) (33270, 3) (133079,) (33270,)


In [82]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 707.0635794152695
R^2 : 0.28865948227097593
        True Labels  Predicted Labels
254319          7.0        185.653768
254320         10.0        180.278017
254321          0.0        180.392568
254322        137.0        228.567101
254323          5.0        185.361945
...             ...               ...
287594         22.0        249.057225
287595       1728.0       1978.232018
287596         13.0        190.709046
287597         13.0        196.486707
287598         43.0        233.960635

[33270 rows x 2 columns]


In [83]:
#fill missing values
missingnews = news.loc[news['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingnews[['viewCount','dislikeCount','likeCount']]))
news.loc[news.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### NONPROFIT & ACTIVISM


In [84]:
nonprof = videos1.loc[videos1['Category']=='NONPROFIT & ACTIVISM']

In [85]:
#check null values, need to fill those
print("null comments")
print(len(nonprof.loc[nonprof['commentCount'].isna()]))

null comments
67


In [86]:
nonprof_noNull = nonprof.loc[nonprof['commentCount'].notnull()]
commentX = nonprof_noNull[['viewCount','dislikeCount','likeCount']]
commentY = nonprof_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(19714, 3) (4929, 3) (19714,) (4929,)


In [87]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 758.8464663464863
R^2 : 0.1399779064411465
        True Labels  Predicted Labels
307370          0.0          2.574141
307371          1.0          1.817806
307372          6.0          2.088785
307373          2.0          2.218036
307374          1.0          2.188354
...             ...               ...
312397         21.0         15.402259
312398          1.0          1.684493
312399          1.0          1.678881
312400          0.0          2.270122
312401          4.0          1.668645

[4929 rows x 2 columns]


In [88]:
#fill missing values
missingnonprof = nonprof.loc[nonprof['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingnonprof[['viewCount','dislikeCount','likeCount']]))
nonprof.loc[nonprof.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### PEOPLE & BLOGS

In [89]:
ppl = videos1.loc[videos1['Category']=='PEOPLE & BLOGS']

In [90]:
#check null values, need to fill those
print("null comments")
print(len(ppl.loc[ppl['commentCount'].isna()]))

null comments
1364


In [91]:
ppl_noNull = ppl.loc[ppl['commentCount'].notnull()]
commentX = ppl_noNull[['viewCount','dislikeCount','likeCount']]
commentY = ppl_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(7909, 3) (1978, 3) (7909,) (1978,)


In [92]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 16147.917322988353
R^2 : 0.23519361580728282
        True Labels  Predicted Labels
320352       4889.0      11004.396307
320353       4104.0       6940.620727
320354       4190.0       6503.007271
320355       8933.0      10122.561486
320356       3306.0       6019.211299
...             ...               ...
323241       2349.0       3692.202961
323242       2441.0       2377.448515
323243       1551.0       2705.935619
323244       5928.0       6132.416988
323245      10326.0      21706.857130

[1978 rows x 2 columns]


In [93]:
#fill missing values
missingppl = ppl.loc[ppl['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingppl[['viewCount','dislikeCount','likeCount']]))
ppl.loc[ppl.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### PETS & ANIMALS

In [94]:
pets = videos1.loc[videos1['Category']=='PETS & ANIMALS']

In [95]:
#check null values, need to fill those
print("null comments")
print(len(pets.loc[pets['commentCount'].isna()]))

null comments
180


In [96]:
pets_noNull = pets.loc[pets['commentCount'].notnull()]
commentX = pets_noNull[['viewCount','dislikeCount','likeCount']]
commentY = pets_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(7730, 3) (1933, 3) (7730,) (1933,)


In [97]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 4212.825842966304
R^2 : 0.7059043933173575
        True Labels  Predicted Labels
331465          4.0       -149.338683
331466         65.0         -0.076981
331467          1.0       -152.335339
331468        175.0        191.246310
331469        242.0        245.933226
...             ...               ...
333562         14.0       -114.886496
333563       1377.0       5073.496945
333564          7.0       -150.175837
333565          8.0       -151.008215
333566         21.0       -138.923848

[1933 rows x 2 columns]


In [98]:
#fill missing values
missingpets = pets.loc[pets['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingpets[['viewCount','dislikeCount','likeCount']]))
pets.loc[pets.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### SCIENCE & TECHNOLOGY

In [99]:
sci = videos1.loc[videos1['Category']=='SCIENCE & TECHNOLOGY']

In [100]:
#check null values, need to fill those
print("null comments")
print(len(sci.loc[sci['commentCount'].isna()]))

null comments
268


In [101]:
sci_noNull = sci.loc[sci['commentCount'].notnull()]
commentX = sci_noNull[['viewCount','dislikeCount','likeCount']]
commentY = sci_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(6053, 3) (1514, 3) (6053,) (1514,)


In [102]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 6217.514007970589
R^2 : 0.26980892324317096
        True Labels  Predicted Labels
339910       1462.0       1945.255294
339911       2574.0       3446.250782
339912       1310.0       1784.961135
339913        715.0       1703.890336
339914       1412.0       2644.250991
...             ...               ...
341420       4065.0       2512.354093
341421      15765.0      15320.531214
341422       1978.0       4099.464378
341423       2459.0       4298.672247
341424      23598.0      22328.148959

[1514 rows x 2 columns]


In [103]:
#fill missing values
missingsci = sci.loc[sci['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingsci[['viewCount','dislikeCount','likeCount']]))
sci.loc[sci.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### SPORTS

In [104]:
sports = videos1.loc[videos1['Category']=='SPORTS']

In [105]:
#check null values, need to fill those
print("null comments")
print(len(sports.loc[sports['commentCount'].isna()]))

null comments
409


In [106]:
sports_noNull = sports.loc[sports['commentCount'].notnull()]
commentX = sports_noNull[['viewCount','dislikeCount','likeCount']]
commentY = sports_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(49173, 3) (12294, 3) (49173,) (12294,)


In [107]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 825.3730105740924
R^2 : 0.2879971337724846
        True Labels  Predicted Labels
391061        376.0        238.078077
391062        480.0        296.322795
391063        287.0        211.078372
391064        117.0        198.428792
391065         87.0        214.207308
...             ...               ...
403383          3.0        185.812892
403384          2.0        174.286879
403385          0.0        174.322860
403386          3.0        175.521845
403387         21.0        187.680210

[12294 rows x 2 columns]


In [108]:
#fill missing values
missingsports = sports.loc[sports['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingsports[['viewCount','dislikeCount','likeCount']]))
sports.loc[sports.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### TRAVEL

In [109]:
travel = videos1.loc[videos1['Category']=='TRAVEL']

In [110]:
#check null values, need to fill those
print("null comments")
print(len(travel.loc[travel['commentCount'].isna()]))

null comments
2


In [111]:
travel_noNull = travel.loc[travel['commentCount'].notnull()]
commentX = travel_noNull[['viewCount','dislikeCount','likeCount']]
commentY = travel_noNull['commentCount']
train,test,y_train,y_test=train_test_split(commentX,commentY, test_size=0.2,shuffle=False,random_state = seed)
print(train.shape,test.shape,y_train.shape,y_test.shape)


(4240, 3) (1060, 3) (4240,) (1060,)


In [112]:
model = LinearRegression()
model.fit(train, y_train)
# predicting the  test set results
y_pred = model.predict(test)
print('Root means score', np.sqrt(mean_squared_error(y_test, y_pred)))
print("R^2 :",model.score(test, y_test))
d1 = {'True Labels': y_test, 'Predicted Labels': y_pred}
SK = pd.DataFrame(data = d1)
print(SK)


Root means score 1215.281549319496
R^2 : 0.8140490443883854
        True Labels  Predicted Labels
407630        155.0        221.766369
407631        493.0       1639.608598
407632        263.0        392.415498
407633         70.0        120.890536
407634        298.0        379.355068
...             ...               ...
408685         14.0         53.866335
408686          9.0         48.551048
408687          8.0         50.217108
408688        151.0         71.409624
408689         24.0         54.344044

[1060 rows x 2 columns]


In [113]:
#fill missing values
missingtravel = travel.loc[travel['commentCount'].isna()]
pred = np.maximum(0,model.predict(missingtravel[['viewCount','dislikeCount','likeCount']]))
travel.loc[travel.commentCount.isnull(), 'commentCount'] = pred

C:\Users\Kai Bing\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## Aggregate Results

In [126]:
frames = [auto,comedy,edu,ent,film,game,how,kids,music,news,nonprof,ppl,pets,sci,sports,travel]
pooled_videos_df = pd.concat(frames)

In [127]:
len(pooled_videos_df)

407093

In [129]:
pooled_videos_df.drop(columns = ['Category','Channel Name'], inplace=True)

In [131]:
pooled_videos_df.shape

(407093, 10)

In [133]:
pooled_videos_df[['viewCount','dislikeCount','likeCount','commentCount']].isnull().values.any()

False

In [132]:
#write csv 
pooled_videos_df.to_csv('pooled_videos_df.csv',index = False, header = True)